In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn import preprocessing
import numpy as np
import statsmodels.api as sm

from stats_utils import MLR

df = pd.read_csv("../data/WHO-SIMPLE.csv")

## Here we are cleaning the data to grab information specifically for the United States, referencing factors that have less than 50% missing data

In [3]:
usa = df[df['Country Code'] == "USA"]

In [5]:
usa = usa.dropna(thresh=30, axis=1)

In [6]:
usa.describe()

,MDG_0000000001,WHS4_100,WHS4_544,WHS8_110,Year,GDP
count,60.000000,40.000000,40.000000,40.000000,60.000000,6.000000e+01
mean,11.928015,94.550000,91.475000,91.625000,1989.500000,7.457538e+12
std,6.373582,2.926011,5.373188,3.780432,17.464249,6.306322e+12
min,5.557430,83.000000,72.000000,82.000000,1960.000000,5.433000e+11
25%,6.798510,94.000000,90.750000,90.750000,1974.750000,1.649989e+12
50%,9.534520,95.000000,93.000000,92.000000,1989.500000,5.802362e+12
75%,16.125405,96.000000,94.000000,92.250000,2004.250000,1.241946e+13
max,25.887410,97.000000,97.000000,98.000000,2019.000000,2.143323e+13


In [8]:
usa.head()

,Country_Year,MDG_0000000001,WHS4_100,WHS4_544,WHS8_110,Country Code,Year,GDP
10619,USA_1960,25.88741,NaN,NaN,NaN,USA,1960.0,5.433000e+11
10620,USA_1961,25.39648,NaN,NaN,NaN,USA,1961.0,5.633000e+11
10621,USA_1962,24.89482,NaN,NaN,NaN,USA,1962.0,6.051000e+11
10622,USA_1963,24.37521,NaN,NaN,NaN,USA,1963.0,6.386000e+11
10623,USA_1964,23.82979,NaN,NaN,NaN,USA,1964.0,6.858000e+11


In [21]:
explanatory_vars = ['WHS4_100', 'WHS4_544', 'WHS8_110', 'GDP']
response_var = 'MDG_0000000001'
x_and_y_cols = explanatory_vars.copy()
x_and_y_cols.insert(0, response_var)

In [22]:
x_and_y_cols

['MDG_0000000001', 'WHS4_100', 'WHS4_544', 'WHS8_110', 'GDP']

## MLR Requires there to be no rows with missing data.  This will result in us being unable to use data from 1950-1979. However, the only factor we have with data that far back is GDP, which would render MLR useless, since there would only be one factor.  We should end up with 40 observations

In [15]:
usa = usa.dropna(how='any')

## Here we are centering the data around the mean for each column, and scaling it to be consistent across all factors.  It retains the integrity of the data and its distribution, and allows us to meet the assumptions of MLR

In [26]:
#center the variables
centered = preprocessing.scale(usa[x_and_y_cols], with_mean='True', with_std='False')

#convert back into a Pandas dataframe and add column names
usa_centered = pd.DataFrame(centered, columns=x_and_y_cols)

In [29]:
MLR(usa_centered, response_var, explanatory_vars)

MLR Results using Sci-kit Learn:
Intercept: 
 -1.402591079336851e-16
Coefficients: 
 [-0.61632755  0.79930072 -0.03473434 -0.9640516 ]

                            OLS Regression Results                            
Dep. Variable:         MDG_0000000001   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     146.4
Date:                Tue, 30 Mar 2021   Prob (F-statistic):           2.45e-21
Time:                        16:13:45   Log-Likelihood:                0.75436
No. Observations:                  40   AIC:                             8.491
Df Residuals:                      35   BIC:                             16.94
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.

(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 0     1.646982
 1     1.633050
 2     1.759371
 3     1.485100
 4     1.413291
 5     1.431641
 6     1.326152
 7     1.416692
 8     1.199980
 9     0.901160
 10    0.830533
 11    0.622470
 12    0.331292
 13    0.249766
 14   -0.399698
 15   -0.063829
 16    0.283102
 17   -0.019975
 18   -0.115624
 19   -0.367315
 20   -0.041986
 21   -0.251263
 22   -0.163706
 23   -0.551155
 24   -0.534929
 25   -0.672047
 26   -0.659822
 27   -0.773209
 28   -0.668995
 29   -0.540758
 30   -0.656007
 31   -0.816633
 32   -0.647783
 33   -0.761684
 34   -1.107088
 35   -1.231204
 36   -1.167784
 37   -1.310913
 38   -1.436997
 39   -1.570176
 dtype: float64)

## THE BELOW CODE IS RELEVANT BUT IS FOR EXPLORATION AND POTENTIAL LATER USE. THE CODE ABOVE IS WHAT WE NEED NOW

In [ ]:
haveNAN

In [3]:
usa = usa[usa['Year'] >= 2000]

In [6]:
usa = usa.dropna(thresh=5, axis=1)

In [9]:
usa = usa.drop(['PCV3', 'ROTAC'], axis=1)

In [11]:
usa = usa.dropna(how='any')

In [13]:
usa.columns

Index(['Country_Year', 'MDG_0000000001', 'WHS4_100', 'WHS4_117', 'WHS4_129',
       'WHS4_544', 'WHS8_110', 'MCV2', 'LBW_NUMBER', 'LBW_PREVALENCE',
       'MDG_0000000026', 'WSH_SANITATION_SAFELY_MANAGED',
       'GHED_CHEGDP_SHA2011', 'Country Code', 'Year', 'GDP'],
      dtype='object')

In [15]:
explanatory_vars = [
    'WHS4_100', 
    'WHS4_117', 
    'WHS4_129',
    'WHS4_544',
    'WHS8_110',
    'MCV2',
    'LBW_NUMBER',
    'LBW_PREVALENCE',
    'MDG_0000000026',
    'WSH_SANITATION_SAFELY_MANAGED',
    'GHED_CHEGDP_SHA2011',
    'GDP'
]
response_var = 'MDG_0000000001'

MLR(usa, response_var, explanatory_vars)

MLR Results using Sci-kit Learn:
Intercept: 
 84.34212770208713
Coefficients: 
 [ 7.38891990e-03 -3.23413748e-03 -7.13132612e-03 -8.72731954e-03
  1.29458569e-02 -1.54420746e-04  4.35818395e-04  1.32798734e-01
 -1.36600617e-02 -8.88507285e-01 -2.91914218e-02  4.02581614e-14]

                            OLS Regression Results                            
Dep. Variable:         MDG_0000000001   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     626.9
Date:                Fri, 26 Mar 2021   Prob (F-statistic):           4.20e-07
Time:                        11:48:45   Log-Likelihood:                 48.430
No. Observations:                  16   AIC:                            -74.86
Df Residuals:                       5   BIC:                            -66.36
Df Model:                          10                                         
Covariance T

C:\Users\Daniel\AppData\Local\conda\conda\envs\Research\lib\site-packages\scipy\stats\stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 10659    7.102501
 10660    7.048594
 10661    6.928501
 10662    6.885447
 10663    6.811521
 10664    6.743192
 10665    6.677381
 10666    6.569558
 10667    6.469457
 10668    6.361755
 10669    6.233547
 10670    6.146232
 10671    6.034202
 10672    5.974738
 10673    5.876853
 10674    5.807188
 dtype: float64)

## The code below eliminates columns with less than 500 actual values, and rows with less than 52.7% cells populated

In [ ]:
#Examine data frame and determine which year has the most complete data set
df.head()
df.describe()

df.groupby(['Year']).count().sum(axis=1)
df = df.dropna(thresh=500, axis=1)


df['SUM'] = df.count(axis=1)
df = df[df['SUM'] >= 16]

## Below is MLR Code, we must first preprocess

In [ ]:
# def scatterplot_2vars(df, x, y):
#     plt.scatter(df[x], df[y], color='green')
#     plt.title('{} Vs {}'.format(x,y), fontsize=14)
#     plt.xlabel(x, fontsize=14)
#     plt.ylabel(y, fontsize=14)
#     plt.grid(True)
#     plt.show()

# def histogram(df, x):
#     plt.hist(df[x], bins="auto", range=(0,df[x].max()))
#     plt.title('{}'.format(x), fontsize=14)
#     plt.xlabel(x, fontsize=14)
#     plt.grid(True)
#     plt.show()

In [ ]:
# #This is to examine if the relationships are linear. Not all are, but many are good! A few may need a 
# # transformation (ie GDP) and a few may not work ultimately
# columns_of_interest = ['PCV3',#'ROTAC','NUTRITION_564',
#                        'WHS4_100','WHS4_117','WHS4_129','WHS4_543','WHS4_544','WHS8_110',
#                        'MCV2','WHS4_128','LBW_NUMBER','LBW_PREVALENCE',
#                        #'NUTRITION_HA_2','NUTRITION_WA_2',#'NUTRITION_WH2','NUTRITION_WH_2','WHOSIS_000006',
#                        'MDG_0000000026','WSH_SANITATION_SAFELY_MANAGED',
#                        #'WHS9_95','WHS_PBR','WSH_2','WSH_3',
#                        #'M_Est_smk_curr','M_Est_smk_daily','TOBACCO_0000000192',
#                        'GHED_CHEGDP_SHA2011','GDP']

# df_transformed = df.copy()
# log_col_transform = ['GDP','LBW_NUMBER','LBW_PREVALENCE','MDG_0000000026','GHED_CHEGDP_SHA2011']

# for col in columns_of_interest:
#     df_transformed[col] = np.log(df[col])
#     scatterplot_2vars(df_transformed, col, 'MDG_0000000001')
#     histogram(df_transformed, col)

# histogram(df_transformed, 'MDG_0000000001')

In [ ]:
counts = df.count()
counts

In [ ]:
haveNAN = df.columns[df.isnull().any()]


In [ ]:
df.describe()

In [ ]:
#Process through all the columns with NaNs:
for feature in haveNAN:
    print(feature)
    bins = df[feature].value_counts(bins=100, sort=False, dropna=False, )
    bins_index = bins.index
    bins_index = bins_index.set_closed("left")
    print("first_interval should be ({},{}, closed='left')\n".format(-1, bins_index[0].left))
    first_interval = pd.Interval(-1, bins_index[0].left, closed='left')
    bins_index = bins_index.insert(0,first_interval)
    df[feature] = df[feature].fillna(-1)
    df[feature] = pd.cut(df[feature], bins=bins_index, right=False)

In [ ]:
df.head()

In [ ]:
df['PCV3'].value_counts()

In [ ]:
#This was good for experimentation, but it creates a factor for every unique value, and since
#the values aren't ordered, it's basically useless besides telling us that -1 is for missing values
codes, uniques = pd.factorize(df['PCV3'])

In [ ]:
codes

In [ ]:
uniques

In [ ]:
x_and_y_cols = columns_of_interest.copy()
x_and_y_cols.append('MDG_0000000001')
df_no_nan = df_transformed[x_and_y_cols].copy().dropna()
X = df_no_nan[columns_of_interest] # Our multiple variables
Y = df_no_nan['MDG_0000000001']

In [ ]:
len(X)


In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

In [ ]:
model.conf_int()